In [2]:
smi_out = []
with open('smi.out') as f:
    smi_out = f.readlines()

In [6]:
smi_out

['Sat May  4 04:54:36 2019       \n',
 '+-----------------------------------------------------------------------------+\n',
 '| NVIDIA-SMI 410.104      Driver Version: 410.104      CUDA Version: 10.0     |\n',
 '|-------------------------------+----------------------+----------------------+\n',
 '| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |\n',
 '| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |\n',
 '|===============================+======================+======================|\n',
 '|   0  GeForce GTX 1070    On   | 00000000:01:00.0  On |                  N/A |\n',
 '|  0%   41C    P8    14W / 200W |   7723MiB /  8118MiB |      0%      Default |\n',
 '+-------------------------------+----------------------+----------------------+\n',
 '                                                                               \n',
 '+-----------------------------------------------------------------------------+\n',
 '| Processes:  

In [55]:
def parse_smi(smi_out):

    # states = ['INIT', 'GPUINFO', 'PROCESSINFO']
    gpu_info = {}
    process_info = []

    GPUINFO_counter = 0 # Help to leave unwanted lines in GPUINFO state 

    curr_state = 'INIT'
    for line in smi_out:
        if curr_state == 'INIT': # Leave out the initial few lines 
            if not line.startswith('|='):
                continue
            else:
                curr_state = 'GPUINFO'
                continue

        if curr_state == 'GPUINFO': # Collect the GPU Util, mem used, total mem
            GPUINFO_counter += 1
            if GPUINFO_counter == 1: # Leave first line in GPUINFO state 
                continue
            if GPUINFO_counter == 2: # Grab info here
                gpu_info = {'GPUUtil':int(line.split()[-3].strip('%')), \
                          'GPUTotalMem':int(line.split()[-5].strip('MiB')), \
                          'GPUUsedMem': int(line.split()[-7].strip('MiB'))}
                continue
            if line.startswith('|='):
                curr_state = 'PROCESSINFO'
                continue

        if curr_state == 'PROCESSINFO':
            if (line.startswith('+')): # Skipping the last line 
                continue
            if line.split()[3] == 'C': # Only care about Compute processes
                line_process_info = {'MemUsed':int(line.split()[5].strip('MiB')), \
                        'ProcessName':line.split()[4], \
                        'ProcessId': int(line.split()[2]), \
                        'GPUId' : int(line.split()[1]) \
                       }
                process_info.append(line_process_info)

    return (gpu_info, process_info)
            


In [56]:
print (parse_smi(smi_out))

({'GPUUtil': 0, 'GPUTotalMem': 8118, 'GPUUsedMem': 7723}, [{'MemUsed': 7355, 'ProcessName': '...nkdesh/virtualenvs/tf_gpu/bin/python3.5', 'GPUId': 0, 'ProcessId': 10997}])
